## ML Project
#### Submitted By:
- **Niv Harel**: 208665869
- **Eytan Muzafi**: 209160308

#### Github: [https://github.com/nivrl/Machine-learning-project]

#### Imports

In [1143]:
import pandas as pd
import math
from datetime import datetime
import numpy as np
import re
import requests
import seaborn as sns
import matplotlib.pyplot as plt

# from sklearn.linear_model import ElasticNet
# from sklearn.model_selection import cross_val_score, GridSearchCV
# from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# import math
# from geopy.geocoders import Nominatim

In [1144]:
#!pip install scikit-learn

#### Reading and understanding the dataset

In [ ]:
Behaviors= pd.read_excel("Behaviors.xlsx",index_col=0)  
Bins= pd.read_excel("Bins.xlsx",index_col=0)  
Infrastructures= pd.read_excel("Infrastructures.xlsx",index_col=0)  
Dirt_Condition= pd.read_excel("Dirt_Condition.xlsx",index_col=0)  

In [ ]:
#Infrastructures.head()

In [ ]:
#Dirt_Condition.head()

In [ ]:
#Dirt_Condition.columns.tolist()

In [ ]:
#Infrastructures.columns.tolist()

In [ ]:
Infrastructures = Infrastructures.reset_index()
Dirt_Condition = Dirt_Condition.reset_index()
Bins = Bins.reset_index()

#### Creating dictionaries for columns renaming 

In [ ]:
bins_mapping = {
    'זמן סיום מהמערכת' : 'End_Time',
    'יישוב': 'City',
    'סמל_יישוב': 'City_Code',
    'שעת הגעה לנקודה': 'Arrival_Time',
    'נ.צ כתובת': 'Coordinate_Address',
    'כתובת תיאור מיקום נקודת המדידה': 'Measurement_Point_Description',
    'סוגנ קודת המדידה': 'Point_Type',
    'האם יש פחים בנקודת המדידה': 'Has_Bins',
    'כמה פחים יש בנקודת המדידה': 'Bin_Count',
    'סוג': 'Type',
    'מצב הפח': 'Bin_Condition',
    'מפלס הפסולת בפח': 'Waste_Level',
    'מפלס הפסולת בפח במספר': 'Waste_Level_Number'
}

dirt_condition_mapping = {
    'זמן סיום מהמערכת' : 'End_Time',
    'יישוב': 'City',
    'סמל יישוב': 'City_Code',
    'שעת הגעה לנקודה': 'Arrival_Time',
    'נ.צ כתובת': 'Coordinate_Address',
    'כתובת תיאור מיקום נקודת המדידה': 'Measurement_Point_Description',
    'סוג נקודת המדידה': 'Point_Type',
    'בדלי סיגריות': 'Cigarette_Butts',
    'קופסאות סיגריות': 'Cigarette_Boxes',
    'מסכות כירורגיות': 'Surgical_Masks',
    'מכלי משקה למיניהם': 'Drink_Containers',
    'פקקים של מכלי משקה': 'Drink_Container_Caps',
    'אריזות מזון Take Away נייר': 'Paper_Takeaway_Packaging',
    'אריזות מזון Take Away פלסטיק': 'Plastic_Takeaway_Packaging',
    'צלחות חדפ': 'Disposable_Plates',
    'סכום חדפ': 'Disposable_Cutlery',
    'כוסות שתייה קרה חדפ': 'Disposable_Cold_Drink_Cups',
    'כוסות שתייה חמה חדפ': 'Disposable_Hot_Drink_Cups',
    'אריזות של חטיפים': 'Snack_Packaging',
    'זכוכית לא מכלי משקה או לא ניתן לזיהוי': 'Non_Bottle_Glass',
    'נייר אחר לא אריזות מזון': 'Other_Paper',
    'פלסטיק אחר שקיות פלסטיק ורכיבי פלסטיק שאינם מכלי משקה אריזות מזון ומשקאות': 'Other_Plastic',
    'פסולת אורגנית': 'Organic_Waste',
    'פסולת בלתי חוקית שקית אשפה מלאה שהושלכה ברחוב': 'Illegal_Dumping',
    'פסולת אחרת למשל בגדים סוללות חומרי פרסום וכל מה שלא קוטלג בקבוצות האחרות': 'Other_Waste',
    'צואת כלבים': 'Dog_Feces',
    'כתמי מסטיק': 'Gum_Stains',
    'פריט פסולת גדול': 'Large_Waste_Item',
    'אריזות קרטון': 'Cardboard_Packaging',
    'גרפיטי': 'Graffiti',
    'אחר1': 'Other1',
    'אחר2': 'Other2',
    'בנקודת המדידה נאספו x שקיות של פרטי לכלוך': 'Bags_Collected',
    'כמה אנשים עברו בנקודת המדידה בזמן התצפית': 'People_Passed_During_Observation'
}

infrastructures_mapping = {
    'זמןסיוםמהמערכת' : 'End_Time',
    'יישוב': 'City',
    'סמל_יישוב': 'City_Code',
    'שעתהגעהלנקודה': 'Arrival_Time',
    'נ.צכתובת': 'Coordinate_Address',
    'כתובתתיאורמיקוםנקודתהמדידה': 'Measurement_Point_Description',
    'התרשמותכלליתמנקודתהמדידה': 'General_Impression',
    'מדרכה': 'Sidewalk',
    'אבנישפה': 'Curb',
    'גדרות': 'Fences',
    'צמחייה': 'Vegetation',
    'הערות': 'Remarks',
    'point_type': 'Point_Type'
}



In [ ]:
bins_renamed = Bins.rename(columns=bins_mapping)
dirt_condition_renamed = Dirt_Condition.rename(columns=dirt_condition_mapping)
infrastructures_renamed = Infrastructures.rename(columns=infrastructures_mapping)

In [ ]:
#bins_renamed

### First Data Cleaning & Organizing 

Take the relevant columns for each DF

In [ ]:
bins_df = bins_renamed[['City_Code','Measurement_Point_Description', 'Point_Type','Bin_Count','Has_Bins','End_Time']]
dirt_condition_df = dirt_condition_renamed.drop(['Arrival_Time','Coordinate_Address','Other1','Other2'], axis=1)
infrastructures_df = infrastructures_renamed.drop(['City', 'Arrival_Time','Coordinate_Address','Remarks'], axis=1)

#### Organizing the PKs for later merge: date, Measurement_Point_Description

In [ ]:
bins_df['Measurement_Point_Description'] = bins_df['Measurement_Point_Description'].str.replace(r'[A-Za-z0-9_\n]', '', regex=True)
dirt_condition_df['Measurement_Point_Description'] = dirt_condition_df['Measurement_Point_Description'].str.replace(r'[A-Za-z0-9_\n]', '', regex=True)
infrastructures_df['Measurement_Point_Description'] = infrastructures_df['Measurement_Point_Description'].str.replace(r'[A-Za-z0-9_\n]', '', regex=True)

In [ ]:
infrastructures_df['End_Time'] = pd.to_datetime(infrastructures_df['End_Time'])
infrastructures_df['Date'] = infrastructures_df['End_Time'].dt.date

dirt_condition_df['End_Time'] = pd.to_datetime(dirt_condition_df['End_Time'])
dirt_condition_df['Date'] = dirt_condition_df['End_Time'].dt.date

bins_df['End_Time'] = pd.to_datetime(bins_df['End_Time'])
bins_df['Date'] = bins_df['End_Time'].dt.date

##### Handling Nulls (pre condition for merging the DFs) - Only for bin_count and has_bins

In [ ]:
avg_bin_count_per_city = (
    bins_df
    .groupby(['City_Code'], as_index=False)
    .agg({'Bin_Count': 'mean'}) ## add avg count of gerbege
)


In [ ]:
avg_bin_count_per_city.isnull().sum()

In [ ]:
# fill nulls with the avg
avg_bin_count_per_city.Bin_Count = avg_bin_count_per_city.Bin_Count.fillna(avg_bin_count_per_city.Bin_Count.mean())
avg_bin_count_per_city.head()

In [ ]:
# round to get integer numbers
## NIV: WITH AI??, why do the round twice? 
avg_bin_count_per_city['Bin_Count'] = avg_bin_count_per_city['Bin_Count'].apply(
    lambda x: math.ceil(x) if x - int(x) > 0.5 else math.floor(x))

In [ ]:
avg_bin_count_per_city.head()

In [ ]:
bins_df['Fixed_Bin_Count'] = bins_df['Bin_Count'].combine_first(bins_df['City_Code'].map(
    avg_bin_count_per_city.set_index('City_Code')['Bin_Count']))

In [ ]:
bins_df.head()
bins_df.shape

In [ ]:
bins_df['Has_Bins'] = bins_df['Has_Bins'].replace('כן', True)
bins_df['Has_Bins'] = bins_df['Has_Bins'].replace('לא', False)
bins_df

In [ ]:
#There is more then one row per pk's
bins_df = (
    bins_df
    .groupby(['City_Code', 'Measurement_Point_Description', 'Point_Type', 'End_Time','Date'], as_index=False)
    .agg({'Has_Bins': 'max','Fixed_Bin_Count': 'mean'}) 
)

In [ ]:
bins_df.shape

In [ ]:
bins_df['Bin_Count'] = bins_df['Fixed_Bin_Count'].apply(
    lambda x: math.ceil(x) if x - int(x) > 0.5 else math.floor(x))

In [ ]:
bins_df.head(10)

#### Merging DFs with inner joins 

In [ ]:
merged_df = pd.merge(infrastructures_df, dirt_condition_df, on=[
    "City_Code", "Measurement_Point_Description", "Date"], how='inner')	

In [ ]:
merged_df.shape

In [ ]:
full_merged_df = pd.merge(merged_df, bins_df, on=
                          ["City_Code", "Measurement_Point_Description", "Date"], how='inner')	

In [ ]:
relevant_columns = ['End_Time_x', 'Date', 'City', 'City_Code', 'Measurement_Point_Description', 'Point_Type_x',
       'Cigarette_Butts', 'Cigarette_Boxes', 'Surgical_Masks', 'Drink_Containers',
       'Drink_Container_Caps', 'Paper_Takeaway_Packaging',
       'Plastic_Takeaway_Packaging', 'Disposable_Plates', 'Disposable_Cutlery',
       'Disposable_Cold_Drink_Cups', 'Disposable_Hot_Drink_Cups',
       'Snack_Packaging', 'Non_Bottle_Glass', 'Other_Paper', 'Other_Plastic',
       'Organic_Waste', 'Illegal_Dumping', 'Other_Waste', 'Dog_Feces',
       'Gum_Stains', 'Large_Waste_Item', 'Cardboard_Packaging', 'Graffiti',
       'Bags_Collected', 'People_Passed_During_Observation', 'Has_Bins','Bin_Count',
       'General_Impression', 'Sidewalk', 'Curb', 'Fences', 'Vegetation']

full_merged_df = full_merged_df[relevant_columns]

In [ ]:
full_merged_df.rename(columns=lambda col: col[:-2] if col.endswith('_x') else col, inplace=True)
full_merged_df.head()

In [ ]:
full_merged_df.shape

#### Fetching dtypes

In [ ]:
full_merged_df.dtypes

In [ ]:
full_merged_df['Date'] = pd.to_datetime(full_merged_df['Date'])

In [ ]:
full_merged_df.dtypes

In [ ]:
for col in full_merged_df.columns:
    if col not in ['Measurement_Point_Description', 'Point_Type']:
        if full_merged_df[col].dtype not in ['int64', 'float64', 'datetime64[ns]']:
            full_merged_df[col] = full_merged_df[col].astype('category')

In [ ]:
full_merged_df.dtypes

In [ ]:
full_merged_df.to_excel('full_merged_df.xlsx', index=False)

### Handling the rest of the Nulls & Outlires

In [ ]:
df= pd.read_excel("full_merged_df.xlsx") 

In [ ]:
df.head()

In [ ]:
len(df.City.unique())

In [ ]:
df.columns

### Create a Grade column for Infrastructure status to idaetify clases acording one numeric column

In [ ]:
for col in ["Sidewalk", "Curb", "Fences", "Vegetation"]:
    print(col)
    print(df[col].unique())
    print(df[col].isnull().sum())
df.General_Impression.isnull().sum()

In [ ]:
df_with_all_nulls = df[df[["Sidewalk", "Curb", "Fences", "Vegetation"]].isnull().all(axis=1)]
df_with_all_nulls.shape
# 21 rows

In [ ]:
status_mapping_dict = {
    'לא תקין (מוזנח)': 3,
    'סביר': 2,
    'תקין (מטופח)': 1,
    'לא רלוונטי':  pd.NA 
}

columns_to_replace = ["Sidewalk", "Curb", "Fences", "Vegetation"]

temp_df = df.copy()
# Replacing values in the specified columns using the mapping dictionary
temp_df['infrastructure_grade']=df[["Sidewalk", "Curb", "Fences", "Vegetation"]].replace(status_mapping_dict).mean(axis=1, skipna=True)


In [ ]:
df_with_nulls = temp_df[temp_df[["Sidewalk", "Curb", "Fences", "Vegetation"]].isnull().all(axis=1)]
df_with_nulls.shape

In [ ]:
## smaller is better
temp_df['infrastructure_grade']

In [ ]:
#Try to more clearly differentiate between infrastructures with different scores, ֿ
#We have highlighted the differences between them to give more meaning to good infrastructures.

status_strong_2_mapping_dict = {
    'לא תקין (מוזנח)': 9,
    'סביר': 4,
    'תקין (מטופח)': 1,
    'לא רלוונטי':  pd.NA 
}

# List of columns where the mapping should be applied
columns_to_replace = ["Sidewalk", "Curb", "Fences", "Vegetation"]

# Replacing values in the specified columns using the mapping dictionary
temp_df['infrastructure_grade_strong'] = df[["Sidewalk", "Curb", "Fences", "Vegetation"]].replace(status_strong_2_mapping_dict).mean(axis=1, skipna=True)


In [ ]:
temp_df.infrastructure_grade.isnull().sum() # 27 rows

In [ ]:
# If all the infrastructure columns ia null or 'לא רלוונטי', we cant calculate the grade for the row. so we decided to drop this rows
temp_df = temp_df.dropna(subset="infrastructure_grade")
temp_df.infrastructure_grade.isnull().sum()

In [ ]:
df = temp_df.drop(["Sidewalk", "Curb", "Fences", "Vegetation","General_Impression"], axis=1)

In [ ]:
plt.figure(figsize=(8, 4))
plt.boxplot(df['infrastructure_grade'], vert=False, patch_artist=True, boxprops=dict(facecolor='lightblue'))
plt.title('Box Plot of infrastructure_grade')
plt.xlabel('infrastructure_grade')
plt.show()

plt.figure(figsize=(8, 4))
plt.boxplot(df['infrastructure_grade_strong'], vert=False, patch_artist=True, boxprops=dict(facecolor='lightblue'))
plt.title('Box Plot of infrastructure_grade_strong')
plt.xlabel('infrastructure_grade_strong')
plt.show()

In [ ]:
# Create equal-width bins (3 bins)
#df['equal_width_bins'] = pd.cut(df['values'], bins=3)

# divide the grade to 3 clases. each class have the same number of rows. because we have alot of rows in good grade
df['depth_bins_infrastructure_grade_strong'], bins = pd.qcut(df['infrastructure_grade_strong'],retbins=True, q=3,labels=False)
df[['depth_bins_infrastructure_grade_strong','infrastructure_grade_strong']]
bins

In [ ]:
df['depth_bins_infrastructure_grade_strong'] = df['depth_bins_infrastructure_grade_strong'].replace(0, 'Great')
df['depth_bins_infrastructure_grade_strong'] = df['depth_bins_infrastructure_grade_strong'].replace(1, 'Good')
df['depth_bins_infrastructure_grade_strong'] = df['depth_bins_infrastructure_grade_strong'].replace(2, 'poor')

In [ ]:
plt.figure(figsize=(8, 4))
sns.scatterplot(
    data=df,
    x=df.index,  # Replace with the desired X-axis if not using the index
    y='infrastructure_grade_strong',
    hue='depth_bins_infrastructure_grade_strong'
)

# Labels and title
plt.title('Scatter Plot: Infrastructure Grade vs Depth Bins', fontsize=10)
plt.xlabel('Index', fontsize=8)  
plt.ylabel('Infrastructure Grade Strong', fontsize=8)
plt.legend(title='Depth Bins Groups', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

### Nulls Handling Division:

### Niv - 
['End_Time', 'Date', 'City', 'City_Code',
       'Measurement_Point_Description', 'Point_Type', 'Cigarette_Butts',
       'Cigarette_Boxes', 'Surgical_Masks', 'Drink_Containers',
       'Drink_Container_Caps', 'Paper_Takeaway_Packaging',
       'Plastic_Takeaway_Packaging', 'Disposable_Plates', 'Disposable_Cutlery',
       'Disposable_Cold_Drink_Cups', 'Disposable_Hot_Drink_Cups',
       'Snack_Packaging', 'Non_Bottle_Glass', 'Other_Paper', 'Other_Plastic']
       
### Mutzafi - 
['Organic_Waste', 'Illegal_Dumping', 'Other_Waste', 'Dog_Feces',
       'Gum_Stains', 'Large_Waste_Item', 'Cardboard_Packaging', 'Graffiti',
       'Bags_Collected', 'People_Passed_During_Observation', 'Has_Bins',
       'Bin_Count', 'General_Impression', 'Sidewalk', 'Curb', 'Fences',
       'Vegetation']


In [ ]:
## Dates Fetching
df['Day_Of_Week'] = df.Date.dt.dayofweek
df['Month'] = df.Date.dt.month

df = df.drop(['End_Time', 'Date'], axis=1)
df[['Month', 'Day_Of_Week']] = df[['Month', 'Day_Of_Week']] = df[['Month', 'Day_Of_Week']].astype('category')

## Fetching Point_type
df[['Point_Type']] = df[['Point_Type']] = df[['Point_Type']].astype('category')


In [ ]:
## City Fetching - Avoiding OverFitting
economical_status = pd.read_excel("economical_status.xlsx")  
economical_status = economical_status[economical_status['Unnamed: 2'].notnull()].reset_index()
economical_status = economical_status.iloc[1:, [2, 7, 9]]
columns_rename_e = ['City_Code','Area_code', 'Economical_Rate']
economical_status.columns = columns_rename_e
economical_status['Economical_Rate'] = economical_status['Economical_Rate'].astype('Float64')
economical_status = economical_status.groupby(['City_Code'], as_index=False).agg({'Economical_Rate': 'mean'})
economical_status

In [ ]:
## City Fetching - Avoiding OverFitting
location_rank = pd.read_excel("location_rank.xlsx")  
location_rank = location_rank.iloc[6:, [3, 9]]
columns_rename_l = ['City_Code','Location_Rate']
location_rank.columns = columns_rename_l
location_rank['Location_Rate'] = location_rank['Location_Rate'].astype('Float64')
location_rank = location_rank.groupby(['City_Code'], as_index=False).agg({'Location_Rate': 'mean'})
location_rank

In [ ]:
len(df)

In [ ]:
## City Fetching - Avoiding OverFitting
df = pd.merge(df, economical_status, on=[
    "City_Code"], how='inner')
df = pd.merge(df, location_rank, on=[
    "City_Code"], how='inner')

In [ ]:
len(df)

In [ ]:
## Street Fetching - Avoiding OverFitting 
def classify_street(street):
    if any(keyword in street for keyword in ['דיין', 'הרצל', 'גוריון', 'רוטשילד', 'בגין', 'דיזינגוף']):
        return 'Main'
    if any(keyword in street for keyword in ['פארק', 'גן']):
        return 'Park'
    if any(keyword in street for keyword in ['קניון', 'מרכז מסחרי', 'שוק']):
        return 'Commercial'
    if any(keyword in street for keyword in ['תחנת דלק', 'תחנת רכבת']):
        return 'Transportation'
    if any(keyword in street for keyword in ['בית ספר', 'מתנ"ס', 'קופת חולים']):
        return 'Public Facility'
    if any(keyword in street for keyword in [
    'מכללת' , 'שדרה', 'דרך', 'מרכז', 'מכללה', 'טיילת', 'כיכר']):  
        return 'Popular'
    return 'Other'

df['Street_Group'] = df['Measurement_Point_Description'].apply(lambda x: classify_street(x))

df.head()

In [ ]:
df['Street_Group'].value_counts() ## Maybe delete most of the others.. We can handle data of 300-400 rows 
## ( by hajaj it need to be greater than 100..)

In [ ]:
df.isnull().sum()

In [ ]:
df['Organic_Waste'] = df.groupby(['City_Code'])['Organic_Waste'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
df.isnull().sum()

In [ ]:
numeric_cols = [col for col in df.columns if df[col].dtype in ["int64","Float64"] and col not in['Economical_Rate', 'Location_Rate']]

In [ ]:
## Examining Outlieres
for col in numeric_cols:
    plt.figure(figsize=(6, 4))
    plt.boxplot(df[col])
    plt.title(f'Boxplot of {col}')
    plt.show() ## Need to be normalized with generic function

### Dimensionality Reduction 

In [ ]:
df = df.drop(['City','Measurement_Point_Description','infrastructure_grade','infrastructure_grade_strong'], axis=1)

In [ ]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df_numeric = df[numeric_cols]

In [ ]:
corr_matrix = df_numeric.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, cmap='coolwarm')
plt.title("Correlation Matrix Heatmap")
plt.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Standardize the data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_numeric)

# Apply PCA
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_scaled)


In [ ]:
df_pca = pd.DataFrame(df_pca, columns=['PC1', 'PC2'])
df_pca

In [ ]:
pca.explained_variance_ratio_

### Encode category columns  

In [ ]:
df.dtypes

In [ ]:
def convert_categorial_by_encoder(X, categorical_cols):
    ohe = OneHotEncoder(drop='first',sparse_output=False) 
    encoded_categorical = ohe.fit_transform(X[categorical_cols])
    encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=ohe.get_feature_names_out(categorical_cols))

    return encoded_categorical_df

In [ ]:
def normlize_nomarical_features(X,numerical_cols):
    for col in numerical_cols:
        X[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    return X[numerical_cols]

In [ ]:
def normalize_and_encode (df_prepared):
    for col in ['Point_Type','City_Code', "Has_Bins","Day_Of_Week","Month","Street_Group"]:
        df[col] = df[col].astype('category')

    X = df_prepared.drop("depth_bins_infrastructure_grade_strong", axis=1)  #  "depth_bins_infrastructure_grade_strong" is the target variable
    y = df_prepared["depth_bins_infrastructure_grade_strong"]
    
    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = X.select_dtypes(include=['category']).columns

    encoded_categorical_df=convert_categorial_by_encoder(X,categorical_cols)
    scaled_numerical_df=normlize_nomarical_features(X,numerical_cols)
    
    # Combine features
    X_processed = pd.concat([encoded_categorical_df, scaled_numerical_df], axis=1)
    
    # Concat 
    normal_df=pd.concat([X_processed, y], axis=1).dropna() 

    return normal_df

In [ ]:
final_df = normalize_and_encode(df)

In [ ]:
final_df.sample(10)